<a href="https://www.kaggle.com/code/agastyapatel/comparing-different-models-types?scriptVersionId=158048177" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture 
!pip install nbdev

# Goal
Notebook tries to compare few model training, explore different training types. We are using high level api 'FastAI' for this purpose. This notebook is based on the quickstart tutorials and tries to explore different models and some training options. Good read for beginners.

## Computer Vision
PET CLASSIFIER using [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/)
### About dataset
- Dataset contains images of cats and dogs
- Contains data of 37 breeds

**Annotation Methods Used**
- .trimaps used for providing hints for separation between foreground and background
- list which provide annotation with index based on categories and filenames
- Name of the file (If Title case then cat else dog)

In [ ]:
# Importing Dependency
from fastai.vision.all import *
petPath = untar_data(URLs.PETS)

In [ ]:
print('BasePath: ',petPath,'\n', petPath.ls())
import os
os.path.exists(path/'images')

In [ ]:
files = get_image_files(path/'images')
print(len(files))   #Number of files
files

In [ ]:
def label_function(f): return 'Cat' if f[0].isupper() else 'Dog'

dls = ImageDataLoaders.from_name_func(path,
                                      files,
                                      label_function,
                                      seed = 0.42,
                                      valid_pct = 0.2,
                                      item_tfms=Resize(224))
dls

In [ ]:
dls.show_batch()

In [ ]:
learn = vision_learner(dls, resnet34, metrics = error_rate)

In [ ]:
learn.fine_tune(2)

In [ ]:
doc(learn.show_results)
learn.show_results(max_n = 3)

### Comparing other pretrained model
We will be using resnet18 and convnext for comparision

In [ ]:
!pip show timm

In [ ]:
import timm
timm.list_models('convnext*') # Prints available model

In [ ]:
learn18 = vision_learner(dls, resnet18, metrics = error_rate)
learn18.fine_tune(2)

In [ ]:
learnCon = vision_learner(dls, 'convnext_nano', metrics = error_rate)
learnCon.fine_tune(2)

In [ ]:
learnCon.show_results(max_n = 3)

Compared different models
- Resnet 34 
- Resnet 18
- ConvNext_Nano

Outcome:
While considering model, following things can be a factor which can be considered : 
- size of the model: Whether the model will fit in gpu while fine tuning?
- Smaller model or less layered Nueral network will be quicker to train but accuracy will be compromised.
- In circumstances where accuracy is primary factor and predication are made extremely critical situation; then more accurate model should be preferred.

## Train Breed Identifier
We will be using from_name_re (identifies label from path using re)

In [ ]:
print(files[0].name)
re = r'^(.*)_\d+.jpg'
breedDls = ImageDataLoaders.from_name_re(path,
                              files,
                              re, item_tfms = Resize(224))
breedDls.show_batch(max_n = 3)

In [ ]:
breedLearner = vision_learner(breedDls, resnet34, metrics = error_rate)       

In [ ]:
breedLearner.fine_tune(2)

In [ ]:
#Breed Learner2 Contains the learning rate suggested by learning rate finder
breedDls = ImageDataLoaders.from_name_re(path,
                              files,
                              re, item_tfms = Resize(224))
breedLearner2 = vision_learner(breedDls, resnet34, metrics = error_rate)
lr = breedLearner2.lr_find(suggest_funcs=(minimum, steep, valley, slide)) 

In [ ]:
breedLearner2.fine_tune(2, lr.slide)

### Resnet34 with Learning Rate Performance
Resnet34 with good learning rate is more accurate when trained for same epochs.

In [ ]:
breedLearner2.show_results()

In [ ]:
interp = Interpretation.from_learner(breedLearner2)
interp.plot_top_losses(9, figsize = (15,10))

In [ ]:
classInterp = ClassificationInterpretation.from_learner(breedLearner2)
classInterp.plot_confusion_matrix(figsize=(12,12), dpi=60)

## Single Label Classification
Using Datablock API. These type of classification is used to make single prediction ie, the class of the object.

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock),
                get_items = get_image_files,
                splitter = RandomSplitter(),
                get_y = using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                item_tfms = Resize(460),
                batch_tfms = aug_transforms(size=224))
dls2 = pets.dataloaders(untar_data(URLs.PETS)/'images')

In [ ]:
dls.show_batch(max_n = 9)

## Multi-Label Classification
Classifier for not only guessing the class of the object but also to draw an bounding box around the object recognized.
### About Dataset
Name: Pascal 2007
[Reference](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/)\
Goal: To guess the class of the object and also to draw a visual bounding box around the object.
Classes Determined:
- Person: person
- Animal: bird, cat, cow, dog, horse, sheep
- Vehicle: aeroplane, bicycle, boat, bus, car, motorbike, train
- Indoor: bottle, chair, dining table, potted plant, sofa, tv/monitor

In [ ]:
# Downloading data set
mlcPath = untar_data(URLs.PASCAL_2007, base='/kaggle/temp/')
print(mlcPath/'train.csv')
mlcPath.ls()

In [ ]:
df = pd.read_csv(mlcPath/'train.csv')
print(type(df))
df

In [ ]:
dls = ImageDataLoaders.from_df(df, mlcPath, folder='train', valid_col='is_valid', label_delim=' ',
                               item_tfms=Resize(460), batch_tfms=aug_transforms(size=224))

In [ ]:
dls.show_batch()

In [ ]:
f1_macro = F1ScoreMulti(thresh=0.5, average='macro')
f1_macro.name = 'F1(macro)'
f1_samples = F1ScoreMulti(thresh=0.5, average='samples')
f1_samples.name = 'F1(samples)'
learn = vision_learner(dls, resnet50, metrics=[partial(accuracy_multi, thresh=0.5), f1_macro, f1_samples])

In [ ]:
lr = learn.lr_find(suggest_funcs = (minimum, slide))

In [ ]:
learn.fine_tune(2, lr.slide)

In [ ]:
learn.predict(mlcPath/'train/000005.jpg')

In [ ]:
interp = Interpretation.from_learner(learn)
interp.plot_top_losses(9)